In [73]:

import numpy as np

import pandas as pd
pd.set_option('display.max_columns', 100)

from matplotlib import pyplot as plt
%matplotlib inline 

import seaborn as sns

# Scikit-Learn for Modeling
import sklearn

# Pickle for saving model files
import pickle

# Import Logistic Regression
from sklearn.linear_model import LogisticRegression

# Import RandomForestClassifier and GradientBoostingClassifer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

# Naive Bayes (Gaussian) has been reported as performing well on this dataset
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB

# Function for splitting training and test set
from sklearn.model_selection import train_test_split 

# Function for creating model pipelines
from sklearn.pipeline import make_pipeline

# For standardization
from sklearn.preprocessing import StandardScaler

# Helper for cross-validation
from sklearn.model_selection import GridSearchCV

# Classification metrics (added later)
from sklearn.metrics import roc_curve, auc



This is a classification task. We are trying to predict which loans will default.
Here we follow a standard ML workflow. We split our data into train and tet sets and will parametrize and test the performance of 4 classification algorithms. The 4 algorithms used will be :
* Regularized L1 logistic regression  - penalize the absolute size of model coefficients.
* Regularized L2 logistic regresion -  penalize the squared size of model coefficients.
* Random Forest Classifier
* Gradient Boosting Classifier

The model will give us a probability of whether a loan will default or not.

In [102]:
# Load analytical base table from Module 2
df = pd.read_csv('lc_analytical_base_table.csv', nrows=2000)

In [103]:
df.head().T


0          1         2  \
acc_open_past_24mths                      4.00       1.00      8.00   
annual_inc                           105000.00  115000.00  80000.00   
delinq_2yrs                               1.00       0.00      1.00   
dti                                       6.17       3.94     27.05   
grade                                     4.00       1.00      5.00   
inq_fi                                    2.00       1.00      1.00   
inq_last_12m                              2.00       0.00      0.00   
inq_last_6mths                            1.00       0.00      0.00   
installment                             623.42     375.88   1228.12   
loan_amnt                             17000.00   12000.00  30950.00   
loan_status                               0.00       0.00      0.00   
max_bal_bc                             5666.00    1152.00   7628.00   
mo_sin_old_rev_tl_op                    146.00     124.00    171.00   
mo_sin_rcnt_rev_tl_op                     3.00      27.00     19.00   
mo_sin_rcnt_tl                            3.00      16.00     19.00   
mort_acc                                  1.00       3.00      0.00   
num_accts_ever_120_pd                     0.00       0.00      0.00   
num_actv_bc_tl                            4.00       1.00      7.00   
num_actv_rev_tl                          10.00       2.00     10.00   
num_bc_sats                               6.00       3.00     11.00   
num_bc_tl                                13.00       3.00     14.00   
num_il_tl                                 8.00       5.00      8.00   
num_op_rev_tl                            17.00       5.00     17.00   
num_rev_accts                            27.00       6.00     24.00   
num_rev_tl_bal_gt_0                      10.00       2.00     10.00   
num_sats                                 19.00       7.00     19.00   
num_tl_90g_dpd_24m                        0.00       0.00      0.00   
num_tl_op_past_12m                        3.00       0.00      0.00   
open_acc                                 19.00       7.00     19.00   
open_acc_6m                               2.00       0.00      0.00   
...                                        ...        ...       ...   
open_rv_12m                               3.00       0.00      0.00   
open_rv_24m                               4.00       0.00      6.00   
pct_tl_nvr_dlq                           97.30      85.70     96.90   
pub_rec                                   0.00       0.00      0.00   
pub_rec_bankruptcies                      0.00       0.00      0.00   
revol_bal                             14878.00    2125.00  18571.00   
tax_liens                                 0.00       0.00      0.00   
tot_coll_amt                           1546.00    2902.00      0.00   
tot_cur_bal                           84797.00  128838.00  37656.00   
tot_hi_cred_lim                      137782.00  151700.00  90852.00   
total_acc                                37.00      14.00     32.00   
total_bal_ex_mort                     14878.00    8156.00  37656.00   
total_bal_il                              0.00    6031.00  19085.00   
total_bc_limit                        22800.00    5400.00  40600.00   
total_cu_tl                               3.00       1.00      0.00   
total_il_high_credit_limit                0.00    9550.00  41252.00   
total_rev_hi_lim                      56600.00    7700.00  49600.00   
fico_range_avg                          677.00     707.00    697.00   
earliest_cr_line_years                   18.00      11.00     15.00   
emp_length_years                          1.00      10.00     10.00   
emp_length_years_missing                  0.00       0.00      0.00   
term_years                                3.00       3.00      3.00   
application_type_Individual               1.00       1.00      1.00   
application_type_Joint App                0.00       0.00      0.00   
home_ownership_MORTGAGE                   1.00       1.00      0.00  

In [104]:
# Create separate object for target variable
y = df["loan_status"]

# Create separate object for input features
X = df.drop("loan_status", axis=1)

We will split the train/test set 80/20.  We also stratify to ensure the target feature is balanced in each subset of data.

In [105]:
# Split X and y into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=1234,
                                                    stratify=df["loan_status"])

# Print number of observations in X_train, X_test, y_train, and y_test
print( len(X_train), len(X_test), len(y_train), len(y_test) )


1600 400 1600 400


In [106]:
X_test

acc_open_past_24mths  annual_inc  delinq_2yrs    dti  grade  inq_fi  \
96                    12.0   250000.00          1.0  17.34    1.0     7.0   
1540                   5.0    95000.00          0.0  16.15    4.0     0.0   
148                    8.0    90000.00          0.0  18.83    3.0     0.0   
257                   11.0    27000.00          0.0  34.22    4.0     1.0   
324                    9.0    75000.00          1.0   3.06    2.0     0.0   
1936                   2.0    32000.00          1.0  24.49    2.0     0.0   
1675                   6.0   100000.00          0.0  39.91    1.0     0.0   
36                     5.0    95000.00          1.0  12.99    4.0     0.0   
4                      9.0    80000.00          0.0  21.54    2.0     3.0   
1880                   8.0    57054.00          0.0  11.51    5.0     2.0   
1176                   5.0   100000.00          0.0  12.47    1.0     0.0   
1471                   8.0    70000.00          0.0   8.83    3.0     0.0   
1998                   3.0    23000.00          0.0  98.90    5.0     0.0   
1951                   6.0    44000.00          0.0  21.11    2.0     3.0   
1955                   5.0    48000.00          0.0   2.50    6.0     1.0   
541                    4.0    29000.00          0.0  35.22    2.0     2.0   
525                    1.0    46000.00          0.0  16.51    3.0     0.0   
937                    1.0    48000.00          1.0  11.53    2.0     0.0   
1560                   3.0    38000.00          0.0  21.23    2.0     0.0   
1630                   5.0   106000.00          0.0  12.79    2.0     0.0   
1205                   3.0    38400.00          0.0  20.19    1.0     1.0   
1837                   1.0    85000.00          0.0   9.84    2.0     1.0   
1978                   3.0    75000.00          0.0  13.25    1.0     1.0   
1136                   8.0    65000.00          0.0  28.62    3.0     4.0   
156                    1.0   115000.00          1.0  16.28    5.0     2.0   
570                    1.0    80000.00          2.0  23.06    5.0     0.0   
868                    4.0    44000.00          0.0  15.66    3.0     1.0   
986                    7.0    66000.00          0.0  16.05    2.0     5.0   
510                    4.0    60000.00          0.0   8.14    1.0     1.0   
349                    4.0    50000.00          0.0  21.93    6.0     0.0   
...                    ...         ...          ...    ...    ...     ...   
1347                   5.0    52000.00          0.0  11.26    2.0     0.0   
333                    7.0   120000.00          0.0  10.92    3.0     0.0   
1112                   4.0    30000.00          1.0  22.20    3.0     2.0   
1334                   3.0    45000.00          0.0  11.39    2.0     0.0   
1265                   3.0    83000.00          0.0   6.55    1.0     2.0   
874                    0.0    51000.00          0.0  12.02    4.0     0.0   
138                    3.0    68000.00          0.0   3.32    2.0     0.0   
821                    2.0   115000.00          0.0  22.30    2.0     1.0   
1760                  10.0    98788.00          0.0  14.61    2.0     2.0   
1535                   5.0    65000.00          0.0  26.35    2.0     2.0   
1182                   5.0    41000.00          2.0  11.48    3.0     0.0   
1897                   4.0    56000.00          0.0  27.28    2.0     1.0   
887                    6.0    82000.00          0.0  25.74    3.0     0.0   
755                    1.0    75000.00          0.0  11.41    4.0     1.0   
1788                   5.0    58488.00          0.0  23.43    2.0     2.0   
1654                   0.0    22000.00          0.0   7.64    1.0     0.0   
157                    1.0    60000.00          1.0   9.08    3.0     0.0   
983                   13.0    51000.00          0.0  24.68    3.0     2.0   
761                    1.0    21000.00          0.0  51.89    3.0     0.0   
1610                   5.0    80000.00          0.0   7.94    1.0     1.0   
12

We will use 4 classification algorithms, create pipelines for each and set algorithm hyperparameters.
Note that we standardize the feature values but subtracting means and dividing by stanard deviation across the feature set.

In [107]:
pipelines = {
    'l1' : make_pipeline(StandardScaler(), LogisticRegression(penalty='l1' , random_state=123)),
    'l2' : make_pipeline(StandardScaler(), LogisticRegression(penalty='l2' , random_state=123)),
    'rf' : make_pipeline(StandardScaler(), RandomForestClassifier(random_state=123)),
    'gb' : make_pipeline(StandardScaler(), GradientBoostingClassifier(random_state=123)),
    'gnb' : make_pipeline(GaussianNB()),
    'bnb' : make_pipeline(BernoulliNB()), 
    'mnb' : make_pipeline(MultinomialNB())
}

In [54]:
np.linspace(1e-3, 1e3, 10)

array([1.00000e-03, 1.11112e+02, 2.22223e+02, 3.33334e+02, 4.44445e+02,
       5.55556e+02, 6.66667e+02, 7.77778e+02, 8.88889e+02, 1.00000e+03])

we will test a range of penalties from 0.001 to 1000 for the l1 and l2 algorithms.

In [108]:
# Logistic Regression hyperparameters
l1_hyperparameters = {'logisticregression__C' : np.linspace(1e-3, 1e3, 10),}
l2_hyperparameters = {'logisticregression__C' : np.linspace(1e-3, 1e3, 10),}

# Random Forest hyperparameters
rf_hyperparameters = {
    'randomforestclassifier__n_estimators': [100, 200],
    'randomforestclassifier__max_features': ['auto', 'sqrt', 0.33]
}
#Gradient Boosting hyperparameters
gb_hyperparameters = {
    'gradientboostingclassifier__n_estimators': [100, 200],
    'gradientboostingclassifier__learning_rate': [0.05, 0.1, 0.2],
    'gradientboostingclassifier__max_depth': [1, 3, 5]
}

In [109]:
gnb_hyperparameters = {}
bnb_hyperparameters = {}
mnb_hyperparameters = {}

In [110]:

hyperparameters = {
    'l1' : l1_hyperparameters,
    'l2' : l2_hyperparameters,
    'rf' : rf_hyperparameters,
    'gb' : gb_hyperparameters,
    'gnb': gnb_hyperparameters,
    'bnb': bnb_hyperparameters,
    'mnb': mnb_hyperparameters
}


Use cross validation to tune each model. Here we use 10 fold cross-validation - i.e., we iteratively create 9 folds plus 1 hold-out fold and will average the training score over the 10 hold-out folds. 

In [111]:
fitted_models = {}
algs_to_test = ['gb']

# Loop through model pipelines, tuning each one and saving it to fitted_models
for name, pipeline in pipelines.items():
    # Create cross-validation object from pipeline and hyperparameters
    model = GridSearchCV(pipeline, hyperparameters[name], cv=10, n_jobs=-1)
    
    # Fit model on X_train, y_train
    if name in algs_to_test:
        model.fit(X_train, y_train)
        fitted_models[name] = model
        print(name, 'has been fitted.')

gb has been fitted.


Let's look at the holdout accuracy scores. This is the percent of observations correctly classified by the models. Note that our data is heavily imbalanced with 80% target values equal to loan paid.
Holdout accuracy scores are not robust to imbalanced data sets.

In [112]:
for name, model in fitted_models.items():
    print( name, model.best_score_ )

gb 0.873125


In [113]:
#Predict classes using L1-regularized logistic regression 
pred = fitted_models['gb'].predict(X_test)

# Display first 5 predictions
pred[:1000]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [114]:
# Import confusion_matrix
from sklearn.metrics import confusion_matrix

# Display confusion matrix for y_test and pred
print( confusion_matrix(y_test, pred) )



[[349   0]
 [ 51   0]]



    The first column includes observations that were predicted to be 1.
    The second column includes observations that were predicted to be 0.
    The first row includes observations that were actually 1.
    The second row includes observations that were actually 0.
3625 were true positives (first column, first row)
154 were true negatives (second column, second row)
On the other hand, 1153 were incorrectly classified.
938 were false positives (first column, second row)
115 were false negatives (second column, first row)

In [89]:
# Predict PROBABILITIES using L1-regularized logistic regression
pred = fitted_models['gb'].predict_proba(X_test)

# show the prediction for the positive class, i.e. loan default
pred = [p[1] for p in pred]

# Display first 5 predictions
pred[:10]


[0.07671347672587087,
 0.1063305823695252,
 0.08280947781417335,
 0.15194620197270936,
 0.06134043085475642,
 0.09527862491630931,
 0.16651057582017945,
 0.14259081515781333,
 0.16064559824401195,
 0.08280947781417335]

In [90]:
# Calculate ROC curve from y_test and pred
fpr, tpr, thresholds = roc_curve(y_test, pred)

In [91]:
# Store fpr, tpr, thresholds in DataFrame and display last 10
pd.DataFrame({'FPR': fpr, 'TPR' : tpr, 'Thresholds' : thresholds}).tail(10)

FPR       TPR  Thresholds
56  0.861272  0.851852    0.076713
57  0.867052  0.851852    0.076599
58  0.901734  0.851852    0.074261
59  0.901734  0.888889    0.073634
60  0.919075  0.888889    0.071887
61  0.919075  0.925926    0.069683
62  0.936416  0.925926    0.065780
63  0.953757  0.925926    0.064397
64  0.965318  0.925926    0.063299
65  1.000000  1.000000    0.061340

In [92]:
# Initialize figure
fig = plt.figure(figsize=(8,8))
plt.title('Receiver Operating Characteristic')

# Plot ROC curve
plt.plot(fpr, tpr, label='gb')
plt.legend(loc='lower right')

# Diagonal 45 degree line
plt.plot([0,1],[0,1],'k--')

# Axes limits and labels
plt.xlim([-0.1,1.1])
plt.ylim([-0.1,1.1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()



In [93]:
# Calculate ROC curve
fpr, tpr, thresholds = roc_curve(y_test, pred)

# Calculate AUROC
print( auc(fpr, tpr) )

0.5724684221794047


The AUROC metric is robust against imbalanced classes and tells us the the likelihood that our model can distinguish between a randomly chosen loan that will default versus a randomly chosen loan that will not default.

In [94]:
# Code here
for name, model in fitted_models.items():
    pred = model.predict_proba(X_test)
    pred = [p[1] for p in pred]
    
    fpr, tpr, thresholds = roc_curve(y_test, pred)
    print( name, auc(fpr, tpr) )

gb 0.5724684221794047


In [54]:
# Save winning model as final_model.pkl
with open('final_model.pkl', 'wb') as f:
    pickle.dump(fitted_models['rf'].best_estimator_, f)